**POZNÁMKA: Tento notebook je určený pre platformu Google Colab, ktorá zdarma poskytuje hardvérovú akceleráciu. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook, pomocou lokálnej grafickej karty.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, KBinsDiscretizer
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from torch.utils.data import TensorDataset
import torch.nn as nn
import torch

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
from class_utils.download import download_file_maybe_extract
download_file_maybe_extract("https://www.dropbox.com/s/8s0ivlo9yshhxkn/winequality.zip?dl=1", directory="data/winequality")

# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

## Klasifikácia kvality vína pomocou neurónových sietí

V rámci tohto notebook-u si môžete klasifikáciu pomocou neurónových sietí vyskúšať na príklade ďalšej jednoduchej dátovej množiny.

**Pozor:**  Príklad je čisto ilustračný. Dátová množina je dobre štruktúrovaná (dáta sú rozdelené do stĺpcov s jasným významom atď.), preto by sa na ňu praxi pravdepodobne aplikoval iný model – napr. založený na rozhodovacích stromoch. Umelé neurónové siete a hlboké učenie sa aplikujú skôr v prípade neštruktúrovaných dát, ako je obraz, audio, text a pod.

### Načítanie a predspracovanie dátovej množiny

Načítame dátovú množinu z CSV súboru:



In [ ]:
df = pd.read_csv("data/winequality/winequality-white.csv")
df.head()

Opis dát nájdeme v prípade potreby v priloženom súbore:



In [ ]:
with open("data/winequality/winequality", "r") as file:
    print("".join(file.readlines()))

#### Rozdelenie dátovej množiny

Ďalej pokračujeme rozdelením dátovej množiny. Dáta budeme v tomto prípade deliť nie na dve, ale až na tri časti: na tréningové, validačné a testovacie dáta v pomere 70 : 5 : 25. Validačná dáta budeme používať počas učenia na regularizáciu a výber modelu (detaily nižšie). Stratifikujeme podľa kvality.



In [ ]:
df_train_valid, df_test = train_test_split(df, test_size=0.25,
                                     stratify=df['quality'],
                                     random_state=4)
df_train, df_valid = train_test_split(df_train_valid, test_size=0.05/0.75,
                                     stratify=df_train_valid['quality'],
                                     random_state=4)

#### Selekcia stĺpcov a vytvorenie pipeline objektu

Ako zvyčajne, znovu určíme, ktoré stĺpce sú numerické a ktoré kategorické a vytvoríme pipeline objekt na predspracovanie.



In [ ]:
categorical_inputs = []
numeric_inputs = list(df.columns[:-1])
output = ["quality"]

input_preproc = make_column_transformer(
    (make_pipeline(
        SimpleImputer(strategy='constant', fill_value='MISSING'),
        OneHotEncoder()),
     categorical_inputs),
    
    (make_pipeline(
        SimpleImputer(),
        StandardScaler()),
     numeric_inputs)
)

Dátová množina udáva kvalitu vína na stupnici od 1 po 10 (stĺpec `quality`). Vzhľadom na to, že dáta obsahujú dosť veľa šumu, je táto stupnica až príliš jemná. Prejdeme preto na hrubšiu stupnicu: rozdelíme si vína len na tri stupne kvality, a to automaticky: pomocou transformátora `KBinsDiscretizer` z balíčka `scikit-learn`.



In [ ]:
output_preproc = KBinsDiscretizer(3, encode='ordinal', strategy='quantile')

#### Aplikácia predspracovania

Ďalej aplikujeme na dáta transformátory. Ako zvyčajne, dbáme na to, aby sme pri testovacích – a v našom prípade aj pri validačných – dátach použili funkciu `transform` a nie `fit_transform`.



In [ ]:
X_train = input_preproc.fit_transform(df_train[categorical_inputs+numeric_inputs])
Y_train = output_preproc.fit_transform(df_train[output]).reshape(-1)

X_valid = input_preproc.transform(df_valid[categorical_inputs+numeric_inputs])
Y_valid = output_preproc.transform(df_valid[output]).reshape(-1)

X_test = input_preproc.transform(df_test[categorical_inputs+numeric_inputs])
Y_test = output_preproc.transform(df_test[output]).reshape(-1)

A transformujeme do dátových typov, ktoré očakáva PyTorch: t.j. na 32-bitové float-y (vstupy) a 64-bitové int-y (označenia tried).



In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

X_train = torch.as_tensor(X_train, dtype=torch.float32).to(device)
Y_train = torch.as_tensor(Y_train, dtype=torch.long).to(device)
X_valid = torch.as_tensor(X_valid, dtype=torch.float32).to(device)
Y_valid = torch.as_tensor(Y_valid, dtype=torch.long).to(device)
X_test = torch.as_tensor(X_test, dtype=torch.float32).to(device)
Y_test = torch.as_tensor(Y_test, dtype=torch.long).to(device)

---
### Úloha 1

**Aplikujte na problém klasifikátor na báze neurónovej siete s dropout-om. V rámci tréningovej slučky logujte v každej epoche výsledky na validačnej množine. Výsledky na validačnej množine môžete použiť na ladenie architektúry siete, miery dropouť-u a pod.** 

**Pomôcka: Čo sa týka veľkosti lineárnych vrstiev, môžete začať napr. s hodnotami:** 

* `num_inputs`;
* 64;
* 32;
* 16;
* `num_outputs`;
---


In [ ]:
class Net(nn.Module):
    def __init__(self, num_inputs, num_outputs):
        
    
    # ------




In [ ]:
num_inputs = X_train.shape[1]
num_outputs = len(np.unique(Y_train.cpu()))
model = Net(num_inputs, num_outputs).to(device)

criterion = nn.CrossEntropyLoss()




# ------





In [ ]:
plt.plot(loss_train, label="train")
plt.plot(loss_valid, label="valid")
plt.xlabel("epoch")
plt.ylabel("loss")
plt.grid(ls='--')
plt.legend()

In [ ]:
# accuracy on the train set
model.eval()
with torch.no_grad():
    y_train_logit = model(X_train)
    y_train = y_train_logit.argmax(dim=1)

Y_train_cpu = Y_train.cpu()
y_train_cpu = y_train.cpu()

cm = pd.crosstab(
    output_preproc.inverse_transform(
        Y_train_cpu.reshape(-1, 1)).reshape(-1),
    output_preproc.inverse_transform(
        y_train_cpu.reshape(-1, 1)).reshape(-1),
    rownames=['actual'],
    colnames=['predicted']
)
print(cm)

acc = accuracy_score(Y_train_cpu, y_train_cpu)
print("Accuracy = {}".format(acc))

In [ ]:
# accuracy on the validation set
model.eval()
with torch.no_grad():
    y_valid_logit = model(X_valid)
    y_valid = y_valid_logit.argmax(dim=1)

Y_valid_cpu = Y_valid.cpu()
y_valid_cpu = y_valid.cpu()

cm = pd.crosstab(
    output_preproc.inverse_transform(
        Y_valid_cpu.reshape(-1, 1)).reshape(-1),
    output_preproc.inverse_transform(
        y_valid_cpu.reshape(-1, 1)).reshape(-1),
    rownames=['actual'],
    colnames=['predicted']
)
print(cm)

acc = accuracy_score(Y_valid_cpu, y_valid_cpu)
print("Accuracy on valid = {}".format(acc))

### Vyhodnotenie na testovacej množine

Keď bude hotový váš finálny model, môžete vyhodnotiť jeho úspešnosť na testovacej množine.



In [ ]:
# evaluate on the test set
model.eval()
with torch.no_grad():
    y_test_logit = model(X_test)
    y_test = y_test_logit.argmax(dim=1)

Y_test_cpu = Y_test.cpu()
y_test_cpu = y_test.cpu()

cm = pd.crosstab(
    output_preproc.inverse_transform(
        Y_test_cpu.reshape(-1, 1)).reshape(-1),
    output_preproc.inverse_transform(
        y_test_cpu.reshape(-1, 1)).reshape(-1),
    rownames=['actual'],
    colnames=['predicted']
)
print(cm)

acc = accuracy_score(Y_test_cpu, y_test_cpu)
print("Accuracy on test = {}".format(acc))

### Klasifikácia pomocou XGBoost

Nakoniec, rovnako ako v notebook-u o regresii, keďže tu pracujeme s peknými, štruktúrovanými, tabuľkovými dátami, skúsime si na nich pre porovnanie znovu natrénovať aj XGBoost model. Je pravdepodobné, že výsledky budú porovnateľné alebo lepšie v porovnaní s neurónovou sieťou a to pri zlomku výpočtových nárokov.



In [ ]:
from xgboost import XGBClassifier
X_train_np = X_train.cpu().numpy()
Y_train_np = Y_train.cpu().numpy()
X_test_np = X_test.cpu().numpy()
Y_test_np = Y_test.cpu().numpy()

model = XGBClassifier()
model.fit(X_train_np, Y_train_np);

In [ ]:
y_test = model.predict(X_test_np)
cm = pd.crosstab(Y_test_np.reshape(-1),
                 y_test.reshape(-1),
                 rownames=['actual'],
                 colnames=['predicted'])
print(cm)

print("Accuracy on test: {}.".format(accuracy_score(
    Y_test_np, y_test
)))